In [4]:
import networkx as nx
import pandas as pd
ppi_df = pd.read_csv("/Volumes/my_expansion/ppi_hugo.tsv", sep='\t')

# create an undirected graph
G = nx.Graph()

# add all edges
for _, row in ppi_df.iterrows():
    g1, g2 = row['GeneA'], row['GeneB']
    score = row.get('combined_score', 1.0)
    G.add_edge(g1, g2, weight=score)

print(f"✅ PPI network created: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

nx.write_edgelist(G, "ppi_hugo.edgelist", data=False)

✅ PPI network created: 18540 nodes, 730537 edges


In [4]:
import json

with open("final_100_disease_to_genes.json") as f:
    final_100_disease_to_genes = {
        k: set(v) for k, v in json.load(f).items()
    }

disease_modules = {}

for doid, gene_set in final_100_disease_to_genes.items():
    # only keep nodes of interests
    present_genes = [g for g in gene_set if g in G.nodes]
    
    # cut subgraph
    subG = G.subgraph(present_genes).copy()
    
    disease_modules[doid] = subG
    
    print(f"{doid}: {len(present_genes)} genes → {subG.number_of_nodes()} nodes, {subG.number_of_edges()} edges")

DOID:8577: 571 genes → 571 nodes, 1205 edges
DOID:0060224: 550 genes → 550 nodes, 1052 edges
DOID:1459: 516 genes → 516 nodes, 1053 edges
DOID:7148: 520 genes → 520 nodes, 1046 edges
DOID:10763: 464 genes → 464 nodes, 601 edges
DOID:5844: 432 genes → 432 nodes, 643 edges
DOID:0050589: 418 genes → 418 nodes, 593 edges
DOID:0050425: 396 genes → 396 nodes, 351 edges
DOID:7188: 399 genes → 399 nodes, 663 edges
DOID:3459: 385 genes → 385 nodes, 399 edges
DOID:14221: 372 genes → 372 nodes, 530 edges
DOID:8893: 349 genes → 349 nodes, 471 edges
DOID:6713: 333 genes → 333 nodes, 479 edges
DOID:2513: 308 genes → 308 nodes, 383 edges
DOID:0080208: 285 genes → 285 nodes, 410 edges
DOID:10871: 293 genes → 293 nodes, 363 edges
DOID:12236: 251 genes → 251 nodes, 265 edges
DOID:418: 242 genes → 242 nodes, 316 edges
DOID:12306: 239 genes → 239 nodes, 210 edges
DOID:8567: 233 genes → 233 nodes, 140 edges
DOID:8923: 231 genes → 231 nodes, 173 edges
DOID:289: 220 genes → 220 nodes, 252 edges
DOID:6543: 19

In [5]:
# save each disease module to file

import os

output_dir = "modules"
os.makedirs(output_dir, exist_ok=True)

for doid, subgraph in disease_modules.items():
    path = os.path.join(output_dir, f"{doid.replace(':', '_')}.edgelist")
    nx.write_edgelist(subgraph, path, data=True)

In [6]:
import numpy as np

def get_lcc_size(G, seed_nodes):
    # ge t sugbraph composite by seed_nodes
    g = nx.subgraph(G, seed_nodes)
    if g.number_of_nodes() == 0:
        return 0
    # get all connected components and find the largest one
    components = list(nx.connected_components(g))
    largest_component = max(components, key=len)
    return len(largest_component)

def get_random_comparison(G,gene_set,sims):
    # getting all genes in the network  
    all_genes = G.nodes()
    number_of_seed_genes = len(gene_set & set(all_genes)) 
    l_list  = []
    
    # simulations with randomly distributed seed nodes
    #print ""
    for i in range(1,sims+1):
        # print out status
        if i % 100 == 0:
            sys.stdout.write("> random simulation [%s of %s]\r" % (i,sims))
            sys.stdout.flush()
        # get random seeds
        rand_seeds = set(random.sample(all_genes,number_of_seed_genes))

        # get rand lcc
        lcc = get_lcc_size(G,rand_seeds)
        l_list.append(lcc) 
        
    # get the actual value
    lcc_observed = get_lcc_size(G,gene_set)

    # get the lcc z-score:
    l_mean = np.mean(l_list)
    l_std  = np.std(l_list)
    if l_std == 0:
        z_score = 'not available'
    else:
        z_score = (1.*lcc_observed - l_mean)/l_std
    results_message = """
> Random expectation:
> lcc [rand] = %s
> => z-score of observed lcc = %s
"""%(l_mean,z_score)
    return results_message
    
def calc_single_set_distance(G, gene_set):
    distances = []
    for gene in gene_set:
        min_dists = []
        for other in gene_set:
            if gene != other:
                try:
                    dist = nx.shortest_path_length(G, gene, other)
                    min_dists.append(dist)
                except nx.NetworkXNoPath:
                    continue
        if min_dists:
            distances.append(min(min_dists))
    return np.mean(distances) if distances else np.nan


def main():

    results = []

    # go through every disease
    
    for doid, genes in final_100_disease_to_genes.items():
        valid_genes = set(genes) & set(G.nodes())
        if len(valid_genes) < 2:
            print(f"Skipping {doid}: Not enough genes in network.")
            continue

        # get lcc size
        S = get_lcc_size(G, valid_genes)

        # calculate shortest distance
        d_s = calc_single_set_distance(G, valid_genes)

        # Random simulation and z-score calculation
        sims = 1000
        rand_S = []
        for _ in range(sims):
            rand_genes = np.random.choice(G.nodes(), size=len(valid_genes), replace=False)
            rand_S.append(get_lcc_size(G, rand_genes))
        mean_rand_S = np.mean(rand_S)
        std_rand_S = np.std(rand_S)
        z_score = (S - mean_rand_S) / std_rand_S if std_rand_S != 0 else np.nan

        # save results
        results.append({
            "DOID": doid,
            "S": S,
            "d_s": d_s,
            "z_score": z_score,
            "mean_rand_S": mean_rand_S,
            "std_rand_S": std_rand_S,
            "num_genes": len(valid_genes)
        })
        print(f"Processed {doid}: S={S}, d_s={d_s:.2f}, z={z_score:.2f}")

    # save as CSV
    df = pd.DataFrame(results)
    df.to_csv("localization_results.csv", index=False)
    print("All results saved to localization_results.csv")

if __name__ == "__main__":
    main()

Processed DOID:8577: S=409, d_s=1.26, z=3.52
Processed DOID:0060224: S=399, d_s=1.26, z=3.95
Processed DOID:1459: S=356, d_s=1.27, z=3.33
Processed DOID:7148: S=361, d_s=1.26, z=3.39
Processed DOID:10763: S=275, d_s=1.33, z=1.47
Processed DOID:5844: S=270, d_s=1.32, z=2.55
Processed DOID:0050589: S=265, d_s=1.31, z=2.79
Processed DOID:0050425: S=186, d_s=1.39, z=-0.20
Processed DOID:7188: S=263, d_s=1.29, z=3.57
Processed DOID:3459: S=216, d_s=1.36, z=1.72
Processed DOID:14221: S=220, d_s=1.33, z=2.43
Processed DOID:8893: S=223, d_s=1.34, z=3.55
Processed DOID:6713: S=205, d_s=1.31, z=3.14
Processed DOID:2513: S=186, d_s=1.35, z=3.43
Processed DOID:0080208: S=169, d_s=1.34, z=3.18
Processed DOID:10871: S=171, d_s=1.35, z=3.12
Processed DOID:12236: S=121, d_s=1.41, z=2.01
Processed DOID:418: S=142, d_s=1.38, z=3.46
Processed DOID:12306: S=121, d_s=1.36, z=2.34
Processed DOID:8567: S=77, d_s=1.58, z=0.33
Processed DOID:8923: S=98, d_s=1.45, z=1.43
Processed DOID:289: S=133, d_s=1.35, z=3

In [ ]:
from scipy.stats import norm
import numpy as np
import pandas as pd
import networkx as nx
import random
import json

G = nx.read_edgelist("ppi_hugo.edgelist")

with open("final_100_disease_to_genes.json") as f:
    final_100_disease_to_genes = json.load(f)
final_100_disease_to_genes = {k: set(v) for k, v in final_100_disease_to_genes.items()}

# Load previous results
df = pd.read_csv("localization_results.csv")

# Supplement d_s random test results
ds_rand_means = []
ds_rand_stds = []
ds_z_scores = []


for i, row in df.iterrows():
    doid = row["DOID"]
    d_s_obs = row["d_s"]
    gene_set = final_100_disease_to_genes[doid]
    valid_genes = list(set(gene_set) & set(G.nodes))
    if len(valid_genes) < 2:
        ds_rand_means.append(np.nan)
        ds_rand_stds.append(np.nan)
        ds_z_scores.append(np.nan)
        continue

    rand_dists = []
    all_nodes = list(G.nodes)
    for _ in range(100):  # can increase to 1000 later
        rand_genes = random.sample(all_nodes, len(valid_genes))
        d = []
        for g in rand_genes:
            others = [x for x in rand_genes if x != g]
            lens = [
                nx.shortest_path_length(G, g, o)
                for o in others
                if nx.has_path(G, g, o)
            ]
            if lens:
                d.append(min(lens))
        if d:
            rand_dists.append(np.mean(d))

    if len(rand_dists) == 0:
        ds_rand_means.append(np.nan)
        ds_rand_stds.append(np.nan)
        ds_z_scores.append(np.nan)
        continue

    mu = np.mean(rand_dists)
    sigma = np.std(rand_dists)
    z = (d_s_obs - mu) / sigma if sigma > 0 else np.nan

    ds_rand_means.append(mu)
    ds_rand_stds.append(sigma)
    ds_z_scores.append(z)

# Add to dataframe
df["d_s_rand_mean"] = ds_rand_means
df["d_s_rand_std"] = ds_rand_stds
df["d_s_z_score"] = ds_z_scores

# Save results
df.to_csv("localization_results_with_ds_test.csv", index=False)
print("✅ d_s statistical test completed and saved.")